## Optimum solutions comparison notebook

Notebook to demonstrate how changing input parameters changed the optimum solution found by PROCESS.

This notebook demonstrates how the optimum solution found by `PROCESS` changes as we vary input parameters. We will use the large tokamak example input file to do this. The figure of merit for this example is to minimise the major radius, `rmajor`. 

We use the functionality from `plot_solutions.py` and from `plot_proc.py` to demonstrate this. 

These tools plot the solution vectors (i.e. final values of optimisation parameters) for different runs of `PROCESS`. This allows visual comparisons of different solution points.

It can use different intra-solution optimisation parameter normalisations (e.g. initial value, parameter range) and inter-solution normalisations (e.g. normalise to a certain solution).

### Known Limitations

- The solution vectors (optimisation parameter values at the solution) currently plotted are normalised to the initial point (from the `IN.DAT`) of each solution: each element of the vector is the $x_{final}/x_{initial}$, the `xcmxxx` values in the `MFILE.DAT`. This allows all optimisation parameters to be plotted on the same axis, showing the relative changes from their initial values across multiple solutions.
- Solutions being plotted together must also have the same optimisation parameters.
- The solutions plotted in this example are fictitious.

# Setup

First we need to generate the necessary MFILES to be used in the rest of this notebook. We run the examples in a temporary directory so all the inputs are copied there and the outputs contained there before the directory is removed when the example has finished running. This keeps the examples directory tidy and does not permanently modify any data files. The use of temporary directories is not needed for regular use of PROCESS.

In [ ]:
# Reload Process each time (keep editable install up-to-date)
%load_ext autoreload
%autoreload 2
from pathlib import Path
from shutil import copy

from functions_for_examples import copy_to_temp_dir

from process.main import SingleRun, setup_loggers

# Define project root dir; when running a notebook, the cwd is the dir the notebook is in
PROJ_DIR = Path.cwd().parent

# setup the loggers so that the output is not spammed with model errors/warnings
setup_loggers()

# Define input file name relative to project dir, then copy to temp dir
script_dir = Path("__file__").parent.resolve()
input_rel = script_dir / "data/large_tokamak_IN.DAT"

temp_dir, temp_input_path, temp_dir_path = copy_to_temp_dir(input_rel, PROJ_DIR)

# Run process on an input file in a temporary directory
single_run = SingleRun(temp_input_path.as_posix())
single_run.run()

input_file_2 = script_dir / "data/large_tokamak_varied_min_net_electric_IN.DAT"

copy(PROJ_DIR / input_file_2, temp_dir.name)
temp_input_path2 = temp_dir_path / "large_tokamak_varied_min_net_electric_IN.DAT"

# Run process on an input file in a temporary directory
single_run = SingleRun(temp_input_path2.as_posix())
single_run.run()

# Plot single solution

First we will look at the original large tokamak optimum solution. We will plot its solution, showing optimisation parameters normalised to their initial values.

In [ ]:
from process.io.plot_solutions import (
    RunMetadata,
    plot_mfile_solutions,
)

# Plot the solution
runs_metadata = [
    RunMetadata(temp_dir_path / "large_tokamak_MFILE.DAT", "large tokamak"),
]

# Figure and dataframe returned for optional further modification
fig1, df1 = plot_mfile_solutions(
    runs_metadata=runs_metadata,
    plot_title="Large tokamak solution",
)
df1

# Comparing optimum solutions

Now we will see the effect that varying an input parameter in the large tokamak input file has on the optimum solution found.

Here, the minimum allowable value for net electric power, `p_plant_electric_net_required_mw`, has been changed from 400MW to 200MW, and PROCESS has found a different optimum solution. 

We can plot the two MFILEs together, showing normalised values of the optimisation parameters at the solution points, as well as the objective function values.

In [ ]:
runs_metadata = [
    RunMetadata(temp_dir_path / "large_tokamak_MFILE.DAT", "original"),
    RunMetadata(
        temp_dir_path / "large_tokamak_varied_min_net_electric_MFILE.DAT",
        "changed min net electric",
    ),
]

fig2, df2 = plot_mfile_solutions(
    runs_metadata=runs_metadata,
    plot_title="2 large tokamak solutions",
)
df2

We can compare the inequality constraint equations for both solutions.

In [ ]:
import matplotlib.pyplot as plt

import process.io.mfile as mf
from process.io.plot_proc import plot_inequality_constraint_equations

original_mfile = mf.MFile((temp_dir_path / "large_tokamak_MFILE.DAT").as_posix())
new_mfile = mf.MFile(
    (temp_dir_path / "large_tokamak_varied_min_net_electric_MFILE.DAT").as_posix()
)

f, axs = plt.subplots(1, 2)
axs[0].set_position([0.0, 0.0, 1.2, 1.5])
axs[1].set_position([1.9, 0.0, 1.2, 1.5])
plot_inequality_constraint_equations(axis=axs[0], m_file=original_mfile, scan=-1)
plot_inequality_constraint_equations(axis=axs[1], m_file=new_mfile, scan=-1)
axs[0].set_title("Original optimum solution")
axs[1].set_title("New optimum solution when changing min net electric")
f.suptitle("Inequality Constraint Equations", y=1.6, x=1.4)

To have lower net electric, `PROCESS` has found a solution where:
- the fusion power has dropped, therefore the neutron wall load has gone down
- the toroidal field required has slightly dropped, therefore the case stress limits are lower as less current in the coils is needed 

# Other solution comparison plots

There are some other ways that you can compare solutions in `PROCESS`. We will demonstrate these for the same two MFILEs as above, but you can add in more MFILEs if you want.

Here we refer to the original large tokamak file as `Large tokamak 1`, and the new solution obtained by varying `p_plant_electric_net_required_mw` as `Large tokamak 2`.

## Plot one solution normalised to another

Normalised differences, relative to the a given solution, can also be plotted. 

In [ ]:
runs_metadata = [
    RunMetadata(temp_dir_path / "large_tokamak_MFILE.DAT", "large tokamak 1"),
    RunMetadata(
        temp_dir_path / "large_tokamak_varied_min_net_electric_MFILE.DAT",
        "large tokamak 2",
    ),
]

fig3, df3 = plot_mfile_solutions(
    runs_metadata=runs_metadata,
    plot_title="Large tokamak 2 solution, relative to large tokamak 1",
    normalising_tag="large tokamak 1",
)
df3

## RMS Errors

Plot RMS errors of multiple solutions relative to a reference solution.

In [ ]:
fig5, df5 = plot_mfile_solutions(
    runs_metadata,
    "Large tokamak 2 solution with RMS errors normalised to large tokamak 1",
    normalising_tag="large tokamak 1",
    rmse=True,
)
df5

## Solutions normalised by range

Use `nitvar` values instead; the solution optimisation parameters are normalised to the range of their upper and lower bounds.

In [ ]:
fig6, df6 = plot_mfile_solutions(
    runs_metadata,
    "Large tokamak 2 solution normalised to the range of the optimisation parameters",
    normalisation_type="range",
)
df6

## Actual values

In [ ]:
fig7, df7 = plot_mfile_solutions(
    runs_metadata,
    "Actual values of optimisation parameters for large tokamak 1 and 2 solutions",
    normalisation_type=None,
)
df7